***

# <ins><font size="6"><font color='indianred'>*Sk*</font><font color='cornflowerblue'>ynthetic</font></font></ins> /  <ins><font size="6"><font color='dimgrey'>Sketcherseis</font>



### <font size="4"><font color='indianred'>sketch</font> to <font color='cornflowerblue'>seismic</font></font>
***

In [ ]:
from skynthetic import skyn

In [ ]:
skyn.go('sketches/annot.png', 30, -1e6, 1e6)